## Running QISKit Programs using IBM Data Science Experience

This article shows you how to run [IBM Quantum Experience](https://quantumexperience.ng.bluemix.net/qx) (IBM QX) Jupyter notebooks using [IBM Data Science Experience](https://datascience.ibm.com/) (IBM DSX).

### About IBM Data Science Experience

IBM DSX is an interactive, collaborative, cloud-based environment where data scientists can use multiple tools to activate their insights.  

If you just starting with QISKit and looking for a webhosted Jupyter notebook environment, IBM DSX is an excellent option. It is fairly easy to get your quantum notebook running in the Jupyter notebook environment provided by IBM DSX. Moreover, it provides a platform where you can create your quantum notebooks, invite fellow researchers to collaborate or simply share your work within the community.

### Prerequisites

- You need to signup for the [IBM DSX](https://datascience.ibm.com/). Tip: It is often convenient to just use an IBM ID for signing up. If you do not have one already, [create it first](https://www.ibm.com/account/us-en/signup/register.html).
- You also need an account for [IBM QX](https://quantumexperience.ng.bluemix.net/qx). You can optionally use the IBM ID here as well. This account is required so that you can connect to the API using an 'API TOKEN'. 

### Steps

1. Download [DSX.ipynb](DSX.ipynb)
2. Logon to the [IBM DSX](https://datascience.ibm.com/) with your IBMID.
3. Create a new notebook from your downloaded [DSX.ipynb](DSX.ipynb)
4. Get your IBM QX API token
    - Logon to the [IBM QX](https://quantumexperience.ng.bluemix.net/qx)
    - Click on your username on the top right, and select *My Account*
    - Copy the token displayed there
5. Update the value *`QX_API_TOKEN`* in the code cell with this API token you just copied in previous step.
6. Run the code cell to install QISKit in your DSX account and connect to the IBM QX. 

### Next Steps

Upload other tutorial notebooks into IBM DSX, copy the following *code cell* and paste it at the beginning of the *other* notebooks

In [ ]:
# This cell does some preparatory work to run this Jupyter notebook using 
# IBM Data Science Experience (datascience.ibm.com). 

# --------------------------------------------------------------------------------------------------
# IMPORTANT NOTES: 
# 1) Log on to your QX account and get your API token. You can get it from the Accounts page 
# (the top-right icon on QX website) Copy-Paste the API token below.BE SURE TO ENCLOSE IN QUOTES ("")
# 2) If you are sharing this notebook, MAKE SURE TO REMOVE the API token string before sharing! 
#   Alternatively, you can modify the code below and separately create and save Qconfig.py file with 
#   API token.
# --------------------------------------------------------------------------------------------------
QX_API_TOKEN = "PUT_YOUR_API_TOKEN_HERE"

import sys
import os

try:
    assert(sys.version_info.major > 2)
except AssertionError:          
    print("This code requires Python 3.5 or beyond. Your version: {}.{}".format(sys.version_info.major,sys.version_info.minor))
    raise

# DO NOT CHANGE THE FOLLOWING assertion
try:
    assert(QX_API_TOKEN != "PUT_YOUR_API_TOKEN_HERE")
except AssertionError:
    print("Update the value of QX_API_TOKEN first!")
    raise
    
# importlib.reload is used to reload the Qconfig module. Fixes any update problems with Qconfig.APItoken 
import importlib

# We need visibility to Qconfig.py module add current dir your sys.path
sys.path.append("./")

# Install qiskit package
!pip install qiskit
!rm -rf Qconfig.py 
!touch Qconfig.py

apitoken_str = 'APItoken=\"{}\"'.format(QX_API_TOKEN)
config_str = "config = { \"url\": 'https://quantumexperience.ng.bluemix.net/api'}"

with open('Qconfig.py', 'w') as fil:
    fil.write(apitoken_str + "\n")
    fil.write(config_str + "\n")
    
# Now import all the modules from qiskit such as QuantumProgram, QuantumCircuit, QuantumRegister etc.
# Optionally, you can modify the qiskit import line below to do specific module imports. 
from qiskit import *
import Qconfig

try:
    # Reload Qconfig module again to make sure the APItoken is updated
    importlib.reload(Qconfig)
except NameError as e:
    print("Name error: ", e.args)
except:
    print("unable to reload the module, continuing the execution..")
    
print("Qconfig.APItoken = ", Qconfig.APItoken)